In [1]:
#importation des librairies nécessaires
import pandas as pd
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
import scipy.stats
import seaborn as sns
%matplotlib inline

#import du dataset
df = pd.read_csv(r"en.openfoodfacts.org.products.csv", sep='\t')
#vérification du dataframe
df.head()

C:\Users\sesa638933\Anaconda3\conda\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,8,13,27,28,29,31,47,52,64,75,77) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,abbreviated_product_name,generic_name,...,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,00000000000000225,http://world-en.openfoodfacts.org/product/0000...,nutrinet-sante,1623855208,2021-06-16T14:53:28Z,1623855209,2021-06-16T14:53:29Z,jeunes pousses,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00000000000003429145,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1630483911,2021-09-01T08:11:51Z,1630484064,2021-09-01T08:14:24Z,L.casei,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,Vitória crackers,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000000000031,http://world-en.openfoodfacts.org/product/0000...,isagoofy,1539464774,2018-10-13T21:06:14Z,1539464817,2018-10-13T21:06:57Z,Cacao,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000000000003327986,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1574175736,2019-11-19T15:02:16Z,1624390765,2021-06-22T19:39:25Z,Filetes de pollo empanado,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
#voir nombre de lignes
len(df.index)

1285137

In [3]:
#voir nombre de colonnes
len(df.columns)

186

In [4]:
#une façon plus simple pour vérifier voir lignes et colonnes
df.shape

(1285137, 186)

In [5]:
#vérifier s'il y a des valeurs na
df.isna().sum()

code                        0
url                         0
creator                     1
created_t                   0
created_datetime            0
                       ...   
choline_100g          1285088
phylloquinone_100g    1283440
beta-glucan_100g      1285121
inositol_100g         1285086
carnitine_100g        1285109
Length: 186, dtype: int64

In [6]:
#Eliminer les colonnes ininteressantes pour faciliter le nettoyage
df.drop(df.columns.difference(['code','url','product_name','countries_en','ingredients_text','proteins_100g','energy-kcal_100g','fat_100g']), 1, inplace=True)
#Eliminer les valeurs na pour les colonnes d'interet
df = df[df['fat_100g'].notna()]
df = df[df['proteins_100g'].notna()]
df = df[df['product_name'].notna()]
df = df[df['ingredients_text'].notna()]



In [7]:
df.shape

(532373, 8)

In [8]:
df.duplicated().sum()

2

In [9]:
df['product_name'].duplicated().sum()

145809

In [10]:
df['ingredients_text'].duplicated().sum()

96293

In [11]:
df['url'].duplicated().sum()

2

In [12]:

#Eliminier lignes où les valeurs pour les variables product_name et ingredients_text sont dupliquées 
df.drop_duplicates(subset=['product_name','ingredients_text'], keep='last', inplace = True)

In [13]:
df.shape

(506607, 8)

In [14]:
#changer toutes les valeurs en minuscules
df = df.applymap(lambda s: s.lower() if type(s) == str else s)
#vérifier
df.head()

,code,url,product_name,countries_en,ingredients_text,energy-kcal_100g,fat_100g,proteins_100g
1,00000000000003429145,http://world-en.openfoodfacts.org/product/0000...,l.casei,spain,"leche semidesnatada, azucar 6.9% leche desnata...",NaN,1.4,2.7
6,0000000000100,http://world-en.openfoodfacts.org/product/0000...,moutarde au moût de raisin,france,eau graines de téguments de moutarde vinaigre ...,NaN,8.2,5.1
25,0000000001199,http://world-en.openfoodfacts.org/product/0000...,solène céréales poulet,france,"antioxydant : érythorbate de sodium, colorant ...",219.0,5.9,9.7
33,0000000001663,http://world-en.openfoodfacts.org/product/0000...,crème dessert chocolat,france,"lait entier, sucre, amidon de maïs, cacao, aga...",0.0,0.0,0.0
41,0000000002264,http://world-en.openfoodfacts.org/product/0000...,baguette poitevin,france,"baguette poite vin pain baguette 50,6%: farine...",207.0,6.7,8.9


In [15]:
df.shape

(506607, 8)

In [16]:
df.to_csv(r'C:\Users\sesa638933\Desktop\P2_zeff_jonas\Projet+Python_Dataset_Edstats_csv\cleaned.csv', index = False)
